## Тематическая модель на данных Last.fm

Сначала настраиваем и подключаем нужные библиотеки

In [1]:
import os
import sys
HOME = '/home/vovapolu/Projects/'
BIGARTM_PATH = HOME + 'bigartm/'
BIGARTM_BUILD_PATH = BIGARTM_PATH + 'build/'
sys.path.append(os.path.join(BIGARTM_PATH, 'src/python'))
os.environ['ARTM_SHARED_LIBRARY'] = os.path.join(BIGARTM_BUILD_PATH, 'src/artm/libartm.so')

Теперь создаем батч и map'ы для исполнителей

In [2]:
import artm.messages_pb2, artm.library, uuid
import csv

plays_file = 'usersha1-artmbid-artname-plays.tsv'

batch = artm.messages_pb2.Batch()
batch.id = str(uuid.uuid4())
artist_id_to_name = {}
artists_idxs = {}
artists = []

Читаем файл и строим батч

In [3]:
last_user_id = ''
now_item_id = 0

users_to_handle = 5000

with open(plays_file, 'rb') as tsvin:
    tsvin = csv.reader(tsvin, delimiter='\t')

    field = None

    for row in tsvin:

        if now_item_id > users_to_handle:
            break

        user_id, artist_id, artist_name, plays = row
        if artist_id not in artists:
            artist_id_to_name[artist_id] = artist_name
            artists_idxs[artist_id] = len(artists)
            artists.append(artist_name)

        if user_id != last_user_id:
            item = batch.item.add()
            item.id = now_item_id
            now_item_id += 1

            field = item.field.add()

            last_user_id = user_id

        field.token_id.append(artists_idxs[artist_id])
        field.token_count.append(int(plays))

for artist in artists:
    try:
        ascii_artist = artist.decode('ascii')
    except UnicodeDecodeError:
        print artist
        print "it was not a ascii-encoded unicode string"
        print "added empty string as artist name"
        batch.token.append('')
    else:
        batch.token.append(ascii_artist)

die Ärzte
it was not a ascii-encoded unicode string
added empty string as artist name
blue Öyster cult
it was not a ascii-encoded unicode string
added empty string as artist name
einstürzende neubauten
it was not a ascii-encoded unicode string
added empty string as artist name
björk
it was not a ascii-encoded unicode string
added empty string as artist name
månegarm
it was not a ascii-encoded unicode string
added empty string as artist name
múm
it was not a ascii-encoded unicode string
added empty string as artist name
sigur rós
it was not a ascii-encoded unicode string
added empty string as artist name
röyksopp
it was not a ascii-encoded unicode string
added empty string as artist name
teräsbetoni
it was not a ascii-encoded unicode string
added empty string as artist name
mötley crüe
it was not a ascii-encoded unicode string
added empty string as artist name
björk
it was not a ascii-encoded unicode string
added empty string as artist name
beyoncé
it was not a ascii-encoded unicode str

Запускаем BigArtm

In [4]:
with artm.library.MasterComponent() as master:
    perplexity_score = master.CreatePerplexityScore()
    top_tokens_score = master.CreateTopTokensScore(num_tokens=4)
    model = master.CreateModel(topics_count=100, inner_iterations_count=10)

    for iteration in range(0, 50):
        master.AddBatch(batch)
        master.WaitIdle()
        model.Synchronize()
        print "Iteration#" + str(iteration) + ": Perplexity = %.3f" % perplexity_score.GetValue(model).value

    top_tokens = top_tokens_score.GetValue(model)

artm.library.Visualizers.PrintTopTokensScore(top_tokens)

Iteration#0: Perplexity = 0.000
Iteration#1: Perplexity = 32489.769
Iteration#2: Perplexity = 7711.811
Iteration#3: Perplexity = 3431.771
Iteration#4: Perplexity = 2078.609
Iteration#5: Perplexity = 1498.465
Iteration#6: Perplexity = 1192.273
Iteration#7: Perplexity = 1007.445
Iteration#8: Perplexity = 885.229
Iteration#9: Perplexity = 798.986
Iteration#10: Perplexity = 735.123
Iteration#11: Perplexity = 686.044
Iteration#12: Perplexity = 647.210
Iteration#13: Perplexity = 615.728
Iteration#14: Perplexity = 589.691
Iteration#15: Perplexity = 567.793
Iteration#16: Perplexity = 549.117
Iteration#17: Perplexity = 532.998
Iteration#18: Perplexity = 518.943
Iteration#19: Perplexity = 506.576
Iteration#20: Perplexity = 495.609
Iteration#21: Perplexity = 485.817
Iteration#22: Perplexity = 477.014
Iteration#23: Perplexity = 469.054
Iteration#24: Perplexity = 461.822
Iteration#25: Perplexity = 455.223
Iteration#26: Perplexity = 449.177
Iteration#27: Perplexity = 443.616
Iteration#28: Perplexity

libartm.so: cannot open shared object file: No such file or directory, fall back to ARTM_SHARED_LIBRARY environment variable
